## Computational Chemistry for Experimentalists
## Module 112: NMR

NMR spectroscopy is central to organic chemistry. Here we predict NMR shieldings  

In [15]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

from pyscf.tools import cubegen 
import py3Dmol
import numpy
import tabulate
from pyscf import gto,scf,tdscf,dft,tddft
from pyscf.tools import cubegen 
from pyscf.prop import nmr,ssc
import matplotlib.pyplot as plt

In [2]:
def RDKitToPySCF(rdkitmol,basis='3-21g'):
    elements = [atom.GetSymbol() for atom in rdkitmol.GetAtoms()]
    coordinates = rdkitmol.GetConformer().GetPositions()
    atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]

    pyscf_mole = gto.Mole(basis=basis)
    pyscf_mole.atom = atoms
    pyscf_mole.build();
    return(pyscf_mole)
    

## Part 1: Acetaldehyde proton NMR 

NMR measures the energy levels of a nuclear spin aligned with an external static magnetic field. Normal DFT calculations assume all nuclear spins and external magnetic fields are zero. We compute the NMR shielding as the second derivative of energy with respect to nuclear spin and magnetic field. These should be computed relative to a reference molecule, typically TMS

This first block computes RDKit geometries for acetaldehyde and TMS, converts them to PySCF molecules, and visualizes them 

In [8]:
m0=Chem.MolFromSmiles('C[Si](C)(C)C')
m1=Chem.AddHs(m0)
AllChem.EmbedMolecule(m1)
AllChem.MMFFOptimizeMolecule(m1)
mb1=Chem.MolToMolBlock(m1)
mm1=RDKitToPySCF(m1)

m0=Chem.MolFromSmiles('O=CC')
m2=Chem.AddHs(m0)
AllChem.EmbedMolecule(m2)
AllChem.MMFFOptimizeMolecule(m2)
mb2=Chem.MolToMolBlock(m2)
mm2=RDKitToPySCF(m2)

p=py3Dmol.view(width=600,height=400,viewergrid=(1,2))
p.addModel(mb1,'sdf',viewer=(0,0))
p.addModel(mb2,'sdf',viewer=(0,1))
p.addPropertyLabels("index","",{'showBackground':'false','alignment': 'center'})
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The next block does HF/3-21G calculations at these geometries, and computes absolute NMR shieldings of all nuclei. Arrays sh1 and sh2 contain a Natomx3x3 array with a 3x3  shielding tensor (for magnetic fields and nuclear magnetic dipole moments in x,y,z directions) for each nucleus. In solution, the molecule tumbles relative to the applied field, so we average the x,x y,y and z,z shieldings. 

In [4]:
mf1=scf.RHF(mm1)
mf1.kernel()
mn1=nmr.rhf.NMR(mf1)
sh1=mn1.kernel()
mf2=scf.RHF(mm2)
mf2.kernel()
mn2=nmr.rhf.NMR(mf2)
sh2=mn2.kernel()

converged SCF energy = -445.04310593225

total shielding of atom 0 C
B_x [213.71312585  -0.97038732  -0.77183786]
B_y [ -0.97034024 214.52755844   1.16088565]
B_z [ -0.77183494   1.16090725 213.99175019]
dia-magnetic contribution
B_x [255.73903427  -2.28957398  -1.82139275]
B_y [ -2.2895692  257.66037746   2.73946643]
B_z [ -1.82138089   2.73942375 256.39595743]
para-magnetic contribution
B_x [-42.02590842   1.31918666   1.0495549 ]
B_y [  1.31922896 -43.13281902  -1.57858077]
B_z [  1.04954594  -1.57851649 -42.40420724]

total shielding of atom 1 Si
B_x [ 5.46155497e+02  9.04678584e-05 -3.95696718e-05]
B_y [-2.25267920e-05  5.46155474e+02  1.26983659e-04]
B_z [-2.09490804e-05 -6.07202703e-05  5.46155569e+02]
dia-magnetic contribution
B_x [ 8.72799387e+02 -2.60063177e-05 -8.76544236e-06]
B_y [-2.35671048e-05  8.72799373e+02 -4.24415501e-07]
B_z [-8.16362277e-06  1.48579731e-05  8.72799373e+02]
para-magnetic contribution
B_x [-3.26643890e+02  1.16474176e-04 -3.08042294e-05]
B_y [ 1.0403

dia-magnetic contribution
B_x [400.33459597  10.38762754   1.01544787]
B_y [ 11.31216735 407.18225061   3.00792659]
B_z [  1.20027113   3.08415901 387.98532773]
para-magnetic contribution
B_x [-1094.55403474  -270.19541792    37.39451691]
B_y [ -287.68472124 -1338.83606035  -246.69508467]
B_z [  33.8987044  -248.13715324  -38.37404171]

total shielding of atom 1 C
B_x [-45.43295848  49.97577795  25.86472465]
B_y [ 49.67551808 -30.70648488 -39.63864873]
B_z [ 25.80467151 -39.66345594 137.05518797]
dia-magnetic contribution
B_x [259.46384044   8.77345222  -1.48944467]
B_y [  7.81954702 248.63137341   5.05230443]
B_z [ -1.6800832    4.97366304 221.26247064]
para-magnetic contribution
B_x [-304.89679892   41.20232574   27.35416932]
B_y [  41.85597106 -279.33785829  -44.69095316]
B_z [ 27.48475471 -44.63711897 -84.20728267]

total shielding of atom 2 C
B_x [211.00752032 -13.91561575  -5.52839878]
B_y [ -2.39881535 166.0557169   -2.12784411]
B_z [ -3.22646911  -1.17837556 177.72126004]
dia-m

In [6]:
#First we determine the average magnetic shielding of all TMS protons
numH=0
sumShield=0
for i in range(mm1.natm):
    if(mm1.atom_charge(i)==1):
        numH=numH+1
        val=(sh1[i,0,0]+sh1[i,1,1]+sh1[i,2,2])/3
        sumShield=sumShield+val
        #print('%2d %2d %6.2f'%(i,mm1.atom_charge(i),val))
aveShield=sumShield/numH
print('TMS proton absolute isotropic shielding:',aveShield)

TMS proton absolute isotropic shielding: 33.66906030607128


In [7]:
# Next we determine the chemical shifts of each acetaldehyde proton. 
print('Shielding of each acetaldehyde proton:\nIndex  Shielding (ppm)')
for i in range(mm2.natm):
    if(mm2.atom_charge(i)==1):
        val=(sh2[i,0,0]+sh2[i,1,1]+sh2[i,2,2])/3
        print('%2d %6.2f '%(i,aveShield-val))

Shielding of each acetaldehyde proton:
Index  Shielding (ppm)
 3  10.98 
 4   2.48 
 5   2.48 
 6   2.86 


The experimental acetaldehyde NMR has a peak around 2.2 ppm integrated intensity 3x, and a peak around 9.8 ppm integrated intensity 1x. Based on the picture above, atoms 4-6 are the methyl protons and atom 3 is the aldehyde proton. Recall from Module 4 that the methyl proton shifts are rotationally averaged at normal temperatures, where methyl rotation is fast on the NMR timescale. 

## Part 2: Spin-Spin Coupling 

Hyperfine interactions spin-spin coupling are useful for structure assignment. Here we compute the 13C-1H spin-spin coupling of isotopically labeled methane. Experimentally, the methane proton at 0.23 ppm is split into a doublet with width 125 Hz, see https://pubs.acs.org/doi/10.1021/ct050083f . The splitting  is independent of the external magnetic field strength, thus different NMR spectrometers will give  different looking spectra. For more complicated molecules, spin-spin couplings must be thermally averaged. 

In [28]:
m0=Chem.MolFromSmiles('C')
m3=Chem.AddHs(m0)
AllChem.EmbedMolecule(m3)
AllChem.MMFFOptimizeMolecule(m3)
mb3=Chem.MolToMolBlock(m3)
mm3=RDKitToPySCF(m3)
p=py3Dmol.view(width=200,height=200)
p.addModel(mb3,'sdf')
p.addPropertyLabels("index","",{'showBackground':'false','alignment': 'center'})
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [29]:
mf3=scf.RHF(mm3)
mf3.kernel()
mn3=nmr.rhf.NMR(mf3)
sh3=mn3.kernel()

converged SCF energy = -39.9766425103058

total shielding of atom 0 C
B_x [2.10696908e+02 3.72204253e-06 2.29063878e-06]
B_y [3.72204090e-06 2.10696914e+02 2.58090528e-06]
B_z [2.29063863e-06 2.58090300e-06 2.10696901e+02]
dia-magnetic contribution
B_x [ 2.50912620e+02 -1.13066381e-07 -6.08877838e-07]
B_y [-1.13066395e-07  2.50912620e+02 -7.18770828e-07]
B_z [-6.08878057e-07 -7.18771094e-07  2.50912621e+02]
para-magnetic contribution
B_x [-4.02157120e+01  3.83510891e-06  2.89951662e-06]
B_y [ 3.83510730e-06 -4.02157057e+01  3.29967611e-06]
B_z [ 2.89951669e-06  3.29967410e-06 -4.02157198e+01]

total shielding of atom 1 H
B_x [30.52445223 -2.34988648 -0.93833497]
B_y [-2.34988679 36.93581178  2.86720895]
B_z [-0.93833491  2.8672084  30.90032595]
dia-magnetic contribution
B_x [27.29907142 -3.37489801 -1.34763348]
B_y [-3.37489855 36.50703888  4.11787368]
B_z [-1.34763356  4.11787328 27.83889881]
para-magnetic contribution
B_x [3.22538082 1.02501153 0.40929851]
B_y [ 1.02501175  0.4287729

In [25]:
print('Shielding of each methane proton:\nIndex  Shielding (ppm)')
for i in range(mm3.natm):
    if(mm3.atom_charge(i)==1):
        val=(sh3[i,0,0]+sh3[i,1,1]+sh3[i,2,2])/3
        print('%2d %6.2f '%(i,aveShield-val))

Shielding of each methane proton:
Index  Shielding (ppm)
 1   0.88 
 2   0.88 
 3   0.88 
 4   0.88 


In [26]:
ms3=ssc.RHF(mf3)
ss3=ms3.kernel()

Reduced spin-spin coupling constant K (Hz)
               #0        #1        #2        #3        #4   
        0 C    0.00000
        1 H   15.24907   0.00000
        2 H   15.24907  -0.84181   0.00000
        3 H   15.24907  -0.84181  -0.84181   0.00000
        4 H   15.24908  -0.84181  -0.84181  -0.84181   0.00000
Spin-spin coupling constant J (Hz)
               #0        #1        #2        #3        #4   
        0 C    0.00000
        1 H  119.65818   0.00000
        2 H  119.65818 -26.26440   0.00000
        3 H  119.65818 -26.26440 -26.26440   0.00000
        4 H  119.65821 -26.26439 -26.26439 -26.26439   0.00000


The 13C-1H spin-spin coupling is computed as 119 Hz, close to the 125 Hz seen experimentally. 

## Part 3: Method and Basis Set 

NMR shifts are sensitive to details of method and basis set. You should try on your own to compute the methane chemical shift and 13C-1H SSCC with different DFT methods and basis sets. 

Your homework for this module is to compute the  B3LYP/6-31G(d) proton NMR spectra of 3-Dimethylaminoacrolein at low and high temperatures. You'll need to determine which protons to rotationally average. You should compare your results to the experiments at http://www1.chem.umn.edu/groups/taton/chem8361/Handouts/9_26.pdf . For 50 points extra credit, report the predicted 1H NMR spectrum of 13C-labeled methane as measured with the TCU NMR, including both the chemical shifts and the spin-spin couplings. 
